In [1]:
import os
import errno

import traceback
import json

import numpy as np

import deepcell
import matplotlib.pyplot as plt

from deepcell.datasets.tracked import hek293,nih_3t3_bench,nih_3t3,hek293_bench,hela_s3_bench,raw2647_bench
from deepcell.datasets.cytoplasm import hela,nih_3t3,cho
from PIL import Image
import imageio
import cv2
import random

import imgaug.augmenters as iaa

import torch
import torchvision.transforms.functional as TF

from skimage import io
import tifffile as tiff

from sklearn.cluster import KMeans

from skimage.transform import resize

/home/mo/.local/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2023-06-27 14:30:32.469451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mo/anaconda3/envs/cell/lib/python3.9/site-packages/cv2/../../../../lib:
2023-06-27 14:30:32.469481: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/mo/anaconda3/envs/cell/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/mo/anaconda3/envs/cell/lib/python3.9/site-packages/torchvision/image.so: undefined sy

In [2]:
def gt_to_pos(track):
    
    plt.imshow(track[8].squeeze(), cmap='gray')
    plt.colorbar()
    plt.show()
    
    
    resized_tensor = np.zeros((track.shape[0], 150, 150, 1))

    for i in range(track.shape[0]):
        #print(i)
        resized_tensor[i,:,:,0] = resize(track[i,:,:,0], (150, 150))

    track_res = (resized_tensor/np.max(resized_tensor))*np.max(track)
    
    plt.imshow(track_res[8].squeeze(), cmap='gray')
    plt.colorbar()
    plt.show()
    
    black_im = np.zeros_like(track_res,dtype=np.uint8)
    frames = []
    #print('gt',track.shape)
    for k in range(track.shape[0]):
        #print(k)
        unique_values, unique_counts = np.unique(track[k], return_counts=True)
        #print('u',unique_values)
        ind_list=[]
        for j in unique_values:
            #print('un',j,k)
            tolerance = 1e-3 # define a tolerance level for the closeness

            # create a boolean mask of elements in track_res[k] that are close to j
            mask = np.isclose(track_res[k], j, atol=tolerance)

            # get the indices of elements in track_res[k] that satisfy the mask
            indices = np.argwhere(mask)
            #indices = np.argwhere(track_res[k] == j)
            new_ind= indices[:, :-1]
            m_ind= mean_ind(new_ind)
            if j!=0:
                ind_list.append(m_ind)
        im_paint=black_im[k]
        image_tensor = im_paint

        # Define the radius of the ball
        radius = 2

        # Define the color of the ball (white)
        color = (255, 255, 255)

        # Define the list of positions
        positions = ind_list

        # Loop over the positions and draw a circle around each one
        for position in positions:
            # Convert the position to integer coordinates
            try:
                x, y = int(position[0]), int(position[1])

                # Draw a circle with the given radius and color
                for i in range(-radius, radius+1):
                    for j in range(-radius, radius+1):
                        if i**2 + j**2 <= radius**2:
                            try:
                                image_tensor[x+i, y+j, :] = 255
                            except IndexError:
                                pass#print('IndErr',i,j)
            except:
                pass

        # Convert the numpy array to an image and display it
        black_im[k]=image_tensor
        im=image_tensor.squeeze()
        frame = Image.fromarray(im)
        frames.append(frame)

# Save the list of frames as a gif

        #plt.imshow(im, cmap='gray')
        #plt.show()
    #imageio.mimsave('video.gif', frames)
    return black_im,ind_list

    
def mean_ind(ind):
    l=len(ind)
    return np.sum(ind,axis=0)/l


def downsample_save_cyto(v_t,name,pos=False,vid=False,vid_nr=0,alt=0):


    # Load the video tensor
    video_tensor = v_t

    # Define the downsampling factor
    downsample_factor = 3

    # Define the pooling layer
    pooling_layer = torch.nn.MaxPool2d(kernel_size=downsample_factor, stride=downsample_factor)

    # Downsample the video tensor using max pooling
    downsampled_tensor = []
    maxi=np.max(video_tensor)
    for i in range(video_tensor.shape[0]):
        frame = (video_tensor[i, :, :, :]/maxi)*255
        frame = frame.astype('float32')
        frame = TF.to_pil_image(frame.squeeze())
        frame = TF.resize(frame, (150, 150))
        
            
        #print('frame_size',frame.size)
        frame = TF.to_tensor(frame).unsqueeze(0)
        
      
        #frame = pooling_layer(frame)
        downsampled_tensor.append(frame)
    downsampled_tensor = torch.cat(downsampled_tensor, dim=0)
    downsampled_tensor = downsampled_tensor.permute(0, 2, 3, 1).squeeze()
    #print('down_shape',downsampled_tensor.shape)
    # Save every 5 consecutive frames to a file
    num_frames = downsampled_tensor.shape[0]
    for i in range(0, num_frames-5, 1):
        frames_to_save = downsampled_tensor[i:i+5]
        single_frame=downsampled_tensor[i]
        frames_to_save = frames_to_save.numpy()
        #print('f_to_s',frames_to_save.shape)
        path=cons_path(i,name,pos,vid,vid_nr,alt)
        #print('path',path)
        np.save(path, frames_to_save)
        
        

        if pos:
            pos_t=2
            path2=cons_path(i,name,pos_t,vid,vid_nr,alt)
            #print('path2',path2)
            if path2 != 0:
                np.save(path2, single_frame)
        else:
            if random.random() < 1:
                pos_t=2
                path2=cons_path(i,name,pos_t,vid,vid_nr,alt)
                #print('path3',path2)
                if path2 != 0:
                    np.save(path2, single_frame)

            
            
    return 0



def cons_path(i,name,pos,vid,vid_nr,alt):
    #print('pos',pos)
    
    
    if pos==True:
        pre='posit_train/'
            
    elif pos==2:
        #print('pos2',pos)
        if random.random() < 1/10:
            #print('embtest')
            pre='emb_test/'
        else:
            pre='emb_train/'
        j = np.random.randint(1, 100001)
        return pre+name+str(j)+'.npy'
    
    else:
        pre='images_train/'
    
    
    mid=name+'/'
    
    if vid:
        end='vn_'+str(vid_nr)+'_alt'+alt+'_'+str(i)+'.npy'
    else:
        end='alt'+alt+'_'+str(i)+'.npy'
    return pre+mid+end

def verify(path1,path2,single=None):
    
    if single:
        tensor1 = np.load(path1)
        tensor2 = np.load(path2)

        # Create a figure with subplots to display the frames side by side
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 20))

        # Loop over each frame and display them in the subplots
        
        # Display the frame from the first tensor in the left subplot
        im1=axs[0].imshow(tensor1, cmap="gray")
        axs[0].axis("off")
        plt.colorbar(im1, ax=axs[0])

        # Display the frame from the second tensor in the right subplot
        im2=axs[1].imshow(tensor2, cmap="gray")
        axs[1].axis("off")
        plt.colorbar(im2, ax=axs[1])

        # Show the figure with the subplots
        plt.show()
        

    # Load the tensors from the .npy files
    tensor1 = np.load(path1)
    tensor2 = np.load(path2)

    # Create a figure with subplots to display the frames side by side
    fig, axs = plt.subplots(nrows=tensor1.shape[0], ncols=2, figsize=(10, 20))

    # Loop over each frame and display them in the subplots
    for i in range(tensor1.shape[0]):
        # Display the frame from the first tensor in the left subplot
        im1=axs[i, 0].imshow(tensor1[i], cmap="gray")
        axs[i, 0].axis("off")
        plt.colorbar(im1, ax=axs[i, 0])


        # Display the frame from the second tensor in the right subplot
        im2=axs[i, 1].imshow(tensor2[i], cmap="gray")
        axs[i, 1].axis("off")
        plt.colorbar(im2, ax=axs[i, 1])


    # Show the figure with the subplots
    plt.show()
    return 0


def save_vid_tens(tens,pos,name,alt):
    for i in range(tens.shape[0]):
        if pos:
            v_t=gt_to_pos(tens[i])[0]
        else:
            v_t=tens[i]
        downsample_save_cyto(v_t,name,pos=pos,vid=True,vid_nr=i,alt=alt)

def save_im_tens(tens,pos,name,alt):
    if pos:
        v_t=gt_to_pos(tens)[0]
    else:
        v_t=tens
    downsample_save_cyto(v_t,name,pos=pos,alt=alt)

        
def alt(X,y,a=None,t_c=None):
    #alt_s=str(0)+'+'+str(0)
    #return X,y,alt_s 
    
    if t_c == None:
        time_cons=get_bit(0.2)
    else:
        time_cons=t_c
        
    if time_cons==1:
        X,y=shuffle(X,y)
        
    if a == None:
        alt = random.randint(0, 1)
    else:
        alt = a
        
        
    if alt==0:
        X = X
        y = y
        #arr_flipped = np.swapaxes(arr_flipped, 1, 2)
    elif alt==1:
        X = np.fliplr(X)
        y = np.fliplr(y)
        
    else:
        X = np.fliplr(X)
        y = np.fliplr(y)
        X = np.flipud(X)
        y = np.flipud(y)
    alt_s=str(time_cons)+'+'+str(alt)
    return X,y,alt_s      
    
    
    
def shuffle(arr1, arr2):
    print('shuffling')
    # Get the number of samples
    n_samples = arr1.shape[0]

    # Generate a random permutation of indices
    perm = np.random.permutation(n_samples)

    # Shuffle the arrays using the same permutation
    shuffled_arr1 = arr1[perm]
    shuffled_arr2 = arr2[perm]

    return shuffled_arr1, shuffled_arr2



def get_bit(probability):
    if random.random() < probability:
        return 1
    else:
        return 0
    
    
    
def alt_vid(X,y,a=None):
    t_c=get_bit(0.2)
    t_c=0 #####
    if a == None:
        a=random.randint(0, 1)
    X_n= np.zeros_like(X)
    y_n= np.zeros_like(X)
    for i in range(X.shape[0]):
        print('i',i,t_c,a)
        X_n[i],y_n[i],alt_s = alt(X[i],y[i],t_c=t_c,a=a)
    
    return X_n,y_n,alt_s
    
    
def tif_to_vt(path,suff):
    
    path_0=path+suff+'{:03d}'.format(0)+'.tif'
        # read the tif file
    img = tiff.imread(path_0)
    #print('i_shape',img.shape)
    
    image_array = np.zeros((120, img.shape[0], img.shape[1], 1))

    # loop over the tif files
    for i in range(120):
        
        path_r=path+suff+'{:03d}'.format(i)+'.tif'
        # read the tif file
        #print('p',path_r)
        try:
            img_r = tiff.imread(path_r)
        except:
            img_r = np.zeros((img.shape[0], img.shape[1]))
        #print('i_shape',img.shape)
        # add the image to the numpy array
        image_array[i,:,:,0] = img_r
    
    #image_a = np.zeros((120, img.shape[0], img.shape[1], 1))
    nonzero_indices = np.nonzero(np.any(image_array, axis=(1,2,3)))[0]
    crop_im = image_array[nonzero_indices]
    return crop_im

def ctc_2_CNet(path_X,suff_X,path_Y,suff_Y,start_ind,prompt='hela_ctc',alt=0,save_dir='/home/mo/Desktop/IWR/Cell_GT_Proj/CNet_cells/',test=False):
    
    data=[]
    l_ind=start_ind
    for i in range(350):
        
        try:
            output_path_Y_c = save_dir+'source/'+str(start_ind+i)+'contr.png'
            
            path_r_X=path_X+suff_X+'{:03d}'.format(i)+'.tif'
            path_r_Y=path_Y+suff_Y+'{:03d}'.format(i)+'.tif'
            # read the tif file
            #print('p',path_r)
            #try:
            img_r_X = tiff.imread(path_r_X)
            img_r_Y = tiff.imread(path_r_Y)


            max_X=np.max(img_r_X)
            img_r_X = (img_r_X/max_X)*255
            max_y=np.max(img_r_Y)
            img_r_Y = (img_r_Y/max_y)*255
            
            
            if prompt=='fluo_ctc, cell, microscopy image, grayscale':
                
                start_x = np.random.randint(0, img_r_X.shape[1] - 512)
                end_x = start_x + 512
                start_y = np.random.randint(0, img_r_X.shape[0] - 512)
                end_y = start_y + 512

                # Crop the random region from img_r_X
                img_r_X = img_r_X[start_y:end_y, start_x:end_x]

                # Crop the random region from img_r_Y
                img_r_Y = img_r_Y[start_y:end_y, start_x:end_x]
            


            u_val = source2CNet(img_r_Y,alt,prompt,first=True)

     
            img_r_X,img_r_Y=augment_images(img_r_X, img_r_Y,alt=alt)


            #if alt==1:
            #    cv2.imwrite(output_path_Y_c, img_r_Y)



            resized_X = cv2.resize(img_r_X, (512, 512))
            resized_Y = cv2.resize(img_r_Y, (512, 512))

            resized_Y_s = source2CNet(resized_Y,alt,prompt,u_v=u_val,test=test)

            # Specify the output file path
            if test:
                save_dir='/home/mo/Desktop/IWR/Cell_GT_Proj/CNet_cells_rd/'
                
            
            
            output_path_X = save_dir+'target/'+str(start_ind+i)+'.png'
            output_path_Y = save_dir+'source/'+str(start_ind+i)+'.png'
            #output_path_Y_c = save_dir+'source/'+str(start_ind+i)+'contr.png'



            # Save the resized image as PNG
            cv2.imwrite(output_path_X, resized_X)
            cv2.imwrite(output_path_Y, resized_Y_s)
            

            entry = {
            "source": "source/" + str(start_ind+i) + ".png",
            "target": "target/" + str(start_ind+i) + ".png",
            "prompt": prompt
            }

            # Append the entry to the data list
            data.append(entry)

            l_ind=l_ind+1

                # Write the data list as JSON to a file



        except Exception as e:
            if not str(e).startswith("[Errno 2]"):
                 traceback.print_exc()
                    #print("An error occurred:", str(e))

        
        
    output_path = save_dir+"/prompt.json"    
    existing_data = []



    # Read existing data from the JSON file, if it exists
    try:
        with open(output_path, "r") as infile:
            for line in infile:
                line = line.strip()  # Remove leading/trailing whitespaces
                if line:
                    data_j = json.loads(line)
                    existing_data.append(data_j)
    except FileNotFoundError:
        pass

    #print('!!!!!!ex',existing_data)
    #print('!!!!!!dat',data)
    # Append new entries to the existing data
    existing_data=existing_data+data
    #print('!!!!!!ex2',existing_data)

    # Write the updated data to the JSON file
    with open(output_path, "w") as outfile:
        for entry in existing_data:
            #print('ent',entry)
            json.dump(entry, outfile)
            outfile.write('\n')    
        
    return l_ind
    
    
def source2CNet(img,aug,art,first=False,u_v=None,test=False):
    unique_values, unique_counts = np.unique(img, return_counts=True)
    #print('u',unique_values)
    #print('c',unique_counts)
    
 
    if first:
        threshold=1

        unique_values = unique_values[unique_counts > threshold]
        unique_counts = unique_counts[unique_counts > threshold]
        
        return unique_values
    
    else:
        

        #if art == 'fluo_ctc, cell, microscopy image, grayscale' and alt==1:
        #    plt.bar(unique_values[unique_values!=0], unique_counts[unique_values!=0])
            #plt.ylim(top=200)
        #    plt.xlabel('Values')
        #    plt.ylabel('Counts')
        #    plt.title('Histogram')

        #    r=random.uniform(0, 1000000)

        #    plt.savefig('/home/mo/Desktop/IWR/Cell_GT_Proj/CNet_cells/'+str(r)+'_'+str(aug)+'_'+art+'.png')


        unique_values=u_v
        #unique_values, unique_counts = remove_similar_values(unique_values, unique_counts, 1.5)

        ind_list=[]
        for j in unique_values:
            #print('un',j)
            tolerance = 1e-3 # define a tolerance level for the closeness

            # create a boolean mask of elements in track_res[k] that are close to j
            mask = np.isclose(img, j, atol=tolerance)

            # get the indices of elements in track_res[k] that satisfy the mask
            indices = np.argwhere(mask)
            #indices = np.argwhere(track_res[k] == j)
            #print('ind',indices)
            new_ind= indices[:, :-1]
            #print('new_ind',new_ind)
            m_ind= mean_ind(indices)
            if j!=0:
                ind_list.append(m_ind)

        image_tensor =  np.zeros_like(img,dtype=np.uint8)

        # Define the radius of the ball
        radius = 5

        # Define the color of the ball (white)
        #color = (255, 255, 255)

        # Define the list of positions
        positions = ind_list
        #print('ind',ind_list)
        
        if test:
            num = random_number()[1]
            #print(num)
            positions = generate_similar_positions(positions,num)
            

        # Loop over the positions and draw a circle around each one
        for position in positions:
            # Convert the position to integer coordinates
            #try:
            x, y = int(position[0]), int(position[1])

            # Draw a circle with the given radius and color
            for i in range(-radius, radius+1):
                for j in range(-radius, radius+1):
                    if i**2 + j**2 <= radius**2:
                        try:
                            image_tensor[x+i, y+j] = 255
                        except IndexError:
                            pass#print('IndErr',i,j)
            #except: 
               # pass
    
    
    return image_tensor



def augment_images(image_X, image_Y, alt):

    # Rotate the images based on the 'alt' value
    rotate_degrees = alt * 90  # Rotate by 90, 180, 270, or 360 degrees
    aug = iaa.Affine(rotate=rotate_degrees)
    augmented_images = aug(images=[image_X, image_Y])
    
    # Return the augmented images
    augmented_image_X, augmented_image_Y = augmented_images
    return augmented_image_X, augmented_image_Y



def generate_similar_positions(positions,num):
    positions = np.array(positions)
    # Step 1: Calculate pairwise distances
    pairwise_distances = np.linalg.norm(positions[:, None] - positions, axis=2)

    # Step 2: Create an empty list for new positions
    new_positions = []
    
    n_clusters=int(np.sqrt(len(positions)/2))
    n_clusters=int(len(positions)/2)

    # Step 3: Generate random distances with the same distribution
    random_distances = np.random.choice(pairwise_distances.flatten(), size=len(positions)-1)

    # Step 4: Apply KMeans clustering to the positions
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(positions)
    cluster_centers = kmeans.cluster_centers_

    # Step 5-11: Generate new positions
    for random_distance in random_distances:
        # Step 6: Randomly select two cluster centers
        idx1, idx2 = np.random.choice(len(cluster_centers), size=2, replace=False)
        p1, p2 = cluster_centers[idx1], cluster_centers[idx2]

        # Step 7-9: Calculate displacement vector and new position
        displacement = (p2 - p1) / np.linalg.norm(p2 - p1)
        new_position = p1 + displacement * random_distance

        # Step 10: Append new position to the list
        new_positions.append(new_position)

    # Step 11: Convert the list to numpy array
    new_positions = np.array(new_positions)
    
    if num != None and len(new_positions)-num > 2:
        #print('nn1',new_positions,num)
        #new_positions = random.sample(new_positions, len(new_positions))
        #print('nn2',new_positions,num)
        new_positions=new_positions[:num]
        

    return new_positions


def random_number():
    if random.random() < 0.25:  # Random float:  0.0 <= x < 1.0
        return True, random.randint(1, 6)  # Random int: 1 <= x <= 6
    else:
        return False, None


'''    
image = io.imread('/home/mo/Desktop/IWR/Cell_GT_Proj/DIC-C2DH-HeLa/01_GT/TRA/man_track030.tif')
image2 = io.imread('/home/mo/Desktop/IWR/Cell_GT_Proj/DIC-C2DH-HeLa/01/t030.tif')
# Display the image
plt.imshow(image, cmap='gray')
plt.colorbar()
plt.show() 

plt.imshow(image2, cmap='gray')
plt.colorbar()
'''

"    \nimage = io.imread('/home/mo/Desktop/IWR/Cell_GT_Proj/DIC-C2DH-HeLa/01_GT/TRA/man_track030.tif')\nimage2 = io.imread('/home/mo/Desktop/IWR/Cell_GT_Proj/DIC-C2DH-HeLa/01/t030.tif')\n# Display the image\nplt.imshow(image, cmap='gray')\nplt.colorbar()\nplt.show() \n\nplt.imshow(image2, cmap='gray')\nplt.colorbar()\n"

In [3]:
print(stop)


'''
nih_3t3_bench
hek293_bench 
hela_s3_bench 
raw2647_bench
'''

NameError: name 'stop' is not defined

In [ ]:
#filename = 'sample_tracking.trks'
#(X_train, y_train), (X_test, y_test) = nih_3t3_bench.load_tracked_data(filename)
(X_train,y_train),(X_test,y_test) = hela.load_data(path='hela_cytoplasm.npz',test_size=0.01)

In [ ]:
print (X_train[7].shape)
print (y_train.shape)

plt.imshow(X_train[7].squeeze(), cmap='gray')
plt.show()
X_train_a,y_train_a,alt_s=alt(X_train,y_train)
print (y_train_a.shape)
plt.imshow(X_train_a[7].squeeze(), cmap='gray')
plt.show()

print(alt_s)




frames = []
video_tensor=X_train
for i in range(video_tensor.shape[0]):
    # Convert each frame to a Pillow image object
    print(video_tensor[i].shape)
    im=video_tensor[i].squeeze()
    print(im.size)
    frame = Image.fromarray((im * 255 / np.max(im)).astype(np.uint8))
    frames.append(frame)

# Save the list of frames as a gif
imageio.mimsave('video_hela.gif', frames)

print(stop)

In [ ]:
X_train_a,y_train_a,alt_s=alt(X_train,y_train,a=0,t_c=0)

save_im_tens(X_train_a,name='cyt_hela',pos=False,alt=alt_s)
save_im_tens(y_train_a,name='cyt_hela',pos=True,alt=alt_s)

X_train_a,y_train_a,alt_s=alt(X_train,y_train,a=0,t_c=1)

save_im_tens(X_train_a,name='cyt_hela',pos=False,alt=alt_s)
save_im_tens(y_train_a,name='cyt_hela',pos=True,alt=alt_s)

X_train_a,y_train_a,alt_s=alt(X_train,y_train,a=1,t_c=1)

save_im_tens(X_train_a,name='cyt_hela',pos=False,alt=alt_s)
save_im_tens(y_train_a,name='cyt_hela',pos=True,alt=alt_s)

In [ ]:
path_X = ['/home/mo/Desktop/IWR/Cell_GT_Proj/CTC_datasets/Fluo-C2DL-Huh7/01/',
          '/home/mo/Desktop/IWR/Cell_GT_Proj/CTC_datasets/Fluo-C2DL-Huh7/02/']
suff_X = 't' 


path_Y = ['/home/mo/Desktop/IWR/Cell_GT_Proj/CTC_datasets/Fluo-C2DL-Huh7/01_GT/TRA/',
          '/home/mo/Desktop/IWR/Cell_GT_Proj/CTC_datasets/Fluo-C2DL-Huh7/02_GT/TRA/']
suff_Y='man_track'

prompt_a='hela_ctc, cell, microscopy image, grayscale'
prompt_b='fluo_ctc, cell, microscopy image, grayscale'

alt=[0,1,2,3]

l_ind=0

for k in range(len(path_X)):
    for h in range(len(alt)):
        if k < 2:
            prompt_ctc=prompt_a
        else:
            prompt_ctc=prompt_b
        l_ind=ctc_2_CNet(path_X[k],suff_X,path_Y[k],suff_Y,start_ind=l_ind,prompt=prompt_b,alt=alt[h])
        print(l_ind)
        
        
l_ind=0        
for k in range(len(path_X)):
    for h in range(len(alt)):
        if k < 2:
            prompt_ctc=prompt_a
        else:
            prompt_ctc=prompt_b
        l_ind=ctc_2_CNet(path_X[k],suff_X,path_Y[k],suff_Y,start_ind=l_ind,prompt=prompt_b,alt=alt[h],test=True)
        print(l_ind)

In [ ]:
in_path='/home/mo/Desktop/IWR/Cell_GT_Proj/red_dots'
out_path='/home/mo/Desktop/IWR/Cell_GT_Proj/CNet_cells_rd/'


red_dots2Cnet(in_path,out_path)

In [4]:
# Path to the image folder
image_folder = "/home/mo/Desktop/IWR/Cell_GT_Proj/image_log/val_vid"

# Create a new folder for the combined images
output_folder = "/home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid"
os.makedirs(output_folder, exist_ok=True)

# Get a list of all image files in the folder
image_files = [filename for filename in os.listdir(image_folder) if filename.endswith(".png")]

# Process each set of corresponding images
for image_file in image_files:
    # Extract the common identifier from the image filename
    identifier = image_file.split("_gs-", 1)[1].split(".")[0]

    # Construct the filenames of the three corresponding images
    samples_file = os.path.join(image_folder, f"samples_cfg_scale_9.00_gs-{identifier}.png")
    reconstruction_file = os.path.join(image_folder, f"reconstruction_gs-{identifier}.png")
    control_file = os.path.join(image_folder, f"control_gs-{identifier}.png")

    # Open the images
    samples_image = Image.open(samples_file)
    reconstruction_image = Image.open(reconstruction_file)
    control_image = Image.open(control_file)

    # Resize the images by half
    new_size = (samples_image.width // 4, samples_image.height // 4)
    samples_image = samples_image.resize(new_size)
    reconstruction_image = reconstruction_image.resize(new_size)
    control_image = control_image.resize(new_size)

    # Create a new image with the combined images
    combined_image = Image.new("RGB", (new_size[0], new_size[1] * 2))
    combined_image.paste(samples_image, (0, 0))
    #combined_image.paste(reconstruction_image, (0, new_size[1]))
    combined_image.paste(control_image, (0, new_size[1]))

    # Save the combined image in the output folder
    output_file = os.path.join(output_folder, f"combined_{identifier}.png")
    combined_image.save(output_file)

    # Print the saved file path
    print(f"Combined image saved: {output_file}")

Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_009995_e-000033_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_024695_e-000083_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_009701_e-000032_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_005291_e-000017_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_026753_e-000090_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_013523_e-000045_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_020285_e-000068_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_000587_e-000001_b-000000.png
Combined image saved: /h

Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_004997_e-000016_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_006761_e-000022_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_002939_e-000009_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_022931_e-000077_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_023519_e-000079_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_030281_e-000102_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_036161_e-000122_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_029693_e-000100_b-000000.png
Combined image saved: /h

Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_024989_e-000084_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_010877_e-000036_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_020285_e-000068_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_035573_e-000120_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_008525_e-000028_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_003233_e-000010_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_012347_e-000041_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_033809_e-000114_b-000000.png
Combined image saved: /h

Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_041159_e-000139_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_028223_e-000095_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_030281_e-000102_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_027047_e-000091_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_041453_e-000140_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_014111_e-000047_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_032045_e-000108_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_029987_e-000101_b-000000.png
Combined image saved: /h

Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_006467_e-000021_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_032633_e-000110_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_005879_e-000019_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_010877_e-000036_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_040277_e-000136_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_039395_e-000133_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_037043_e-000125_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_037631_e-000127_b-000000.png
Combined image saved: /h

Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_021461_e-000072_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_025283_e-000085_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_003527_e-000011_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_037337_e-000126_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_026753_e-000090_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_010289_e-000034_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_025577_e-000086_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_009407_e-000031_b-000000.png
Combined image saved: /h

Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_040571_e-000137_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_002351_e-000007_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_003233_e-000010_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_014111_e-000047_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_012935_e-000043_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_011759_e-000039_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_004115_e-000013_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_024107_e-000081_b-000000.png
Combined image saved: /h

Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_020579_e-000069_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_004703_e-000015_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_034985_e-000118_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_030281_e-000102_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_024695_e-000083_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_038219_e-000129_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_035279_e-000119_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_040571_e-000137_b-000000.png
Combined image saved: /h

Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_039395_e-000133_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_003821_e-000012_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_032045_e-000108_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_035867_e-000121_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_022343_e-000075_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_025871_e-000087_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_033515_e-000113_b-000000.png
Combined image saved: /home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val_vid/combined_008525_e-000028_b-000000.png
Combined image saved: /h